In [5]:
import math
import datasets
import IBM1 as ibm1

# pretty print variabeles on line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
# Some helper functions to print useful information after each 
# EM iteration

def print_likelihood(i, lprobs, log_likelihood):
    likelihood = math.exp(log_likelihood)
    if i == 1:
        print('iteration  log_likelihood  likelihood')
    print(f'{i} {log_likelihood:.3f} {likelihood:.3f}')

def print_lexicon_probs(i, lprobs, log_likelihood):
    for s in lprobs.keys():
        for t in lprobs[s].keys():
            if lprobs[s][t] > 0:
                print (s, t, lprobs[s][t])
    print()

In [11]:
# Run EM on toy example
s_t_pairs, s_vocabulary, t_vocabulary = datasets.example_data()
s_t_pairs
lprobs = ibm1.EM(s_t_pairs, s_vocabulary, t_vocabulary, 
                 print_likelihood)
lprobs

[(['das', 'Haus'], ['the', 'house']),
 (['das', 'Buch'], ['the', 'book']),
 (['ein', 'Buch'], ['a', 'book'])]

iteration  log_likelihood  likelihood
1 -4.159 0.016
2 -1.151 0.316
3 -0.842 0.431
4 -0.586 0.557
5 -0.390 0.677
6 -0.252 0.777
7 -0.159 0.853
8 -0.100 0.905
9 -0.062 0.940
10 -0.039 0.961
11 -0.025 0.975
12 -0.017 0.983
13 -0.012 0.989
14 -0.008 0.992
15 -0.006 0.994
16 -0.005 0.995
17 -0.004 0.996
18 -0.003 0.997
19 -0.003 0.997
20 -0.002 0.998


defaultdict(<function IBM1._initialize_lexicon_probabilities.<locals>.<lambda>>,
            {'Buch': defaultdict(<function IBM1._initialize_lexicon_probabilities.<locals>.<lambda>.<locals>.<lambda>>,
                         {'a': 9.729458618407092e-06,
                          'book': 0.9999876443477214,
                          'the': 2.6261936603029755e-06}),
             'Haus': defaultdict(<function IBM1._initialize_lexicon_probabilities.<locals>.<lambda>.<locals>.<lambda>>,
                         {'house': 0.9684579278295524,
                          'the': 0.031542072170447605}),
             'das': defaultdict(<function IBM1._initialize_lexicon_probabilities.<locals>.<lambda>.<locals>.<lambda>>,
                         {'book': 2.6261936603029776e-06,
                          'house': 9.729458618407095e-06,
                          'the': 0.9999876443477212}),
             'ein': defaultdict(<function IBM1._initialize_lexicon_probabilities.<locals>.<lambda>.<locals>.<l

In [ ]:
# Run EM on toy example with NULL words
s_t_pairs, s_vocabulary, t_vocabulary = datasets.example_data_null_words()
s_t_pairs
lprobs = ibm1.EM(s_t_pairs, s_vocabulary, t_vocabulary, print_likelihood)
lprobs

# QUESTION: how can likelihood be bigger than 1?
# QUESTION: these NULL behavior is hard to interpret

In [ ]:
# Run EM on training data set
# (...) We are assuming that French words are generated by English 
# words, and that the NULL word is therefore on the English side.

s_t_pairs, s_vocabulary, t_vocabulary = datasets.training_data(
    "training/hansards.36.2.e", "training/hansards.36.2.f")
lprobs = ibm1.EM(s_t_pairs, s_vocabulary, t_vocabulary, print_likelihood)

iteration  log_likelihood  likelihood
1 -34858025.333 0.000
2 -8405523.373 0.000
3 -5424561.055 0.000
4 -4514727.763 0.000
5 -4203772.133 0.000
